In [1]:
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup, Comment
import re
import time

In [80]:
# scraping 1 game
pbp_data = []

game_url = 'https://www.pro-football-reference.com/boxscores/202309070kan.htm'
r = requests.get(game_url)
print(f'status: {r.status_code}')

game_page_soup = BeautifulSoup(r.text)

200
Chiefs vs. Lions 
 total plays: 132


In [102]:
# scraping drive data

# home drives
home_drive_data = []
home_parent = game_page_soup.find('div', {'id': 'all_home_drives'})
if home_parent:
    print('found home_parent')
    home_div = home_parent.find(string=lambda text: isinstance(text, Comment))
    if home_div:
        comment_soup = BeautifulSoup(str(home_div), 'html.parser')
        home_table = comment_soup.find('table', {'id': 'home_drives'})
        if home_table:
            print('found home_table')
            cols = home_table.find('thead').find('tr')
            column_headers = [th.get_text(strip=True) for th in cols.find_all('th')]
            drives = home_table.find('tbody').find_all('tr')
            for drive in drives:
                columns = drive.find_all(['th', 'td'])
                drive_data = [column.get_text(strip=False) for column in columns]
                home_drive_data.append(drive_data)
home_drives = pd.DataFrame(columns = column_headers, data = drive_data)


found home_parent
found home_table


ValueError: Shape of passed values is (8, 1), indices imply (8, 8)

In [101]:

home_table

<table class="sortable stats_table" data-cols-to-freeze=",1" id="home_drives">
<caption>Chiefs Drives Table</caption>
<colgroup><col/><col/><col/><col/><col/><col/><col/><col/></colgroup>
<thead>
<tr>
<th aria-label="#" class="poptip center" data-stat="drive_num" scope="col">#</th>
<th aria-label="Quarter" class="poptip center" data-stat="quarter" scope="col">Quarter</th>
<th aria-label="Time" class="poptip center" data-stat="time_start" scope="col">Time</th>
<th aria-label="LOS" class="poptip center" data-stat="start_at" scope="col">LOS</th>
<th aria-label="Plays" class="poptip center" data-stat="play_count_tip" data-tip="Number of plays in drive.&lt;br /&gt;P - Pass, R - Rush, Y - Penalty" scope="col">Plays</th>
<th aria-label="Length" class="poptip center" data-stat="time_total" scope="col">Length</th>
<th aria-label="Net Yards (Just Plays)" class="poptip center" data-stat="net_yds" scope="col">Net Yds</th>
<th aria-label="Result" class="poptip center" data-stat="end_event" scope="c

In [ ]:
# scraping pbp data
pbp_parent = game_page_soup.find('div', {'class': 'table_wrapper', 'id': 'all_pbp'})
if pbp_parent:
    pbp_div = pbp_parent.find(string=lambda text: isinstance(text, Comment))
    if pbp_div:
        comment_soup = BeautifulSoup(str(pbp_div), 'html.parser')
        pbp_table = comment_soup.find('table', {'id': 'pbp'})
        if pbp_table:
            cols = pbp_table.find('thead').find('tr')
            column_headers = [th.get_text(strip=True) for th in cols.find_all('th')]
            plays = pbp_table.find('tbody').find_all('tr',{'class': ''})
            for play in plays:
                columns = play.find_all(['th', 'td'])
                play_data = [column.get_text(strip=False) for column in columns]
                pbp_data.append(play_data)


pbp_data = pd.DataFrame(columns = column_headers, data = pbp_data)

# Setting up receiving and kicking teams
coin_toss = pbp_data.iloc[0]
pbp_data = pbp_data.drop(0)
coin_toss = coin_toss[7]
teams = re.findall(r'\b[A-Z][a-zA-Z]*\b', coin_toss)
match = re.search(r"(\w+)\s+to\s+receive\s+the\s+opening\s+kickoff", coin_toss)
if match:
    receiving_team = match.group(1)
team1 = teams[0]
team2 = teams[1]

# dropping timeouts
pbp_data = pbp_data[pbp_data['Location'].str.strip() != '']
pbp_data = pbp_data.dropna(subset=['Location'])

print(f'{team1} vs. {team2} \n total plays: {len(pbp_data)}')

In [89]:
# General Cleaning
pbp_data['field_side'] = pbp_data['Location'].str.extract(r'([A-Z]+)')
pbp_data['yardline'] = pbp_data['Location'].str.extract(r'([0-9]+)')
pbp_data['yardline'] = pbp_data['yardline'].astype(int)
pbp_data['minute'] = pbp_data['Time'].str.extract(r'([0-9]+):').astype(int)
pbp_data['seconds'] = pbp_data['Time'].str.extract(r'[0-9]+:([0-9]+)').astype(int)
pbp_data['seconds_ratio'] = (pbp_data['seconds'] / 60).astype(float)
pbp_data['Numeric_time'] = pbp_data['minute'] + pbp_data['seconds_ratio']
pbp_data = pbp_data.drop(columns=['minute', 'seconds', 'seconds_ratio'])
pbp_data


,Quarter,Time,Down,ToGo,Location,DET,KAN,Detail,EPB,EPA,field_side,yardline,Numeric_time
1,1,15:00,,,KAN 35,0,0,"Harrison Butker kicks off 65 yards, touchback.",0.000,0.610,KAN,35,15.000000
2,1,15:00,1,10,DET 25,0,0,David Montgomery up the middle for 7 yards (ta...,0.610,1.010,DET,25,15.000000
3,1,14:29,2,3,DET 32,0,0,Jared Goff pass incomplete short right,1.010,0.300,DET,32,14.483333
4,1,14:25,3,3,DET 32,0,0,Jared Goff pass incomplete deep right intended...,0.300,-1.240,DET,32,14.416667
5,1,14:19,4,3,DET 32,0,0,"Jack Fox punts 61 yards, returned by Richie Ja...",-1.240,0.320,DET,32,14.316667
...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,4,1:53,2,5,KAN 25,21,20,David Montgomery right guard for 3 yards (tack...,3.710,3.400,KAN,25,1.883333
137,4,1:47,3,2,KAN 22,21,20,David Montgomery up the middle for 2 yards (ta...,3.400,4.240,KAN,22,1.783333
139,4,1:42,1,10,KAN 20,21,20,Jared Goff kneels for -1 yards,4.240,3.560,KAN,20,1.700000
140,4,1:05,2,11,KAN 21,21,20,Jared Goff kneels for -1 yards,3.560,2.740,KAN,21,1.083333


In [85]:
pbp_data['Time'].str.extract(r'[0-9]+:([0-9]+)')

,0
1,00
2,00
3,29
4,25
5,19
...,...
135,53
137,47
139,42
140,05
